In [1]:
import pandas as pd
import os

pred_method = "af2_trial"

original_directory = "/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/original_pdbs" ## Native PDB directory
folder_path = f"/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/{pred_method}"
complex_list = [
    f for f in os.listdir(folder_path)
    if os.path.isdir(os.path.join(folder_path, f)) and not f.startswith(".")
]
print(complex_list)  # should print prediction folders ['7ar0_B_A_af2', '...', '...']

## file hierarchy should be like this
# af2
# ├── 7ar0_B_A_af2
# │   ├── 7ar0_B_A_af2.a3m
# │   ├── 7ar0_B_A_af2.png
# │   ├── 7ar0_B_A_af2.png
# │   ├── 7ar0_B_A_af2.png
# │   ├── ...
# ├── 7bnv_H_L_A_af2
# ├── ...

## create results folder
result_folder = pred_method + "_result"
result_path = os.path.join("/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/", result_folder)
os.makedirs(result_path, exist_ok=True)
dockq_output = f"{pred_method}_dockq_fnat_scores.csv"  ## dockq output
pdockq_output = f"{pred_method}_pdockq2_fit.csv" ## pdockq2 output
combined = f"{pred_method}_combined.csv" ### combined results with dockq_pdock2 and model scores

['7bbj_H_L_A_af2', '7ar0_B_A_af2']


In [2]:
## run dockq function
import os
import sys
import csv
import glob
from DockQ.DockQ import load_PDB, run_on_all_native_interfaces
from statistics import mean

def merge_chains(model, chains_to_merge):
    """
    Merges specified chains in the given model.

    Parameters
    ----------
    model : Bio.PDB.Structure
        The model in which the chains are to be merged.
    chains_to_merge : list of str
        The list of chain IDs to be merged.

    Returns
    -------
    model : Bio.PDB.Structure
        The model with the specified chains merged.
    """
    print(f"Merging chains {chains_to_merge} in model")
    for chain in chains_to_merge[1:]:
        for res in list(model[chain]):
            res.id = (chains_to_merge[0], res.id[1], res.id[2])
            model[chains_to_merge[0]].add(res)
        model.detach_child(chain)
    model[chains_to_merge[0]].id = "".join(chains_to_merge)
    return model

def calculate_dockq(model, native, chain_map):
    """
    Calculates DockQ scores for the given model and native structures based on the chain map.

    Parameters
    ----------
    model : Bio.PDB.Structure
        The model structure.
    native : Bio.PDB.Structure
        The native structure.
    chain_map : dict
        The mapping of chains between model and native structures.

    Returns
    -------
    results : dict
        The results containing various DockQ metrics.
    dockq_score : float
        The DockQ score.
    """
    print(f"Calculating DockQ score with chain_map: {chain_map}")
    results, dockq_score = run_on_all_native_interfaces(model, native, chain_map=chain_map)
    return results, dockq_score

def process_models(models):
    """
    Processes the provided models and calculates DockQ scores.

    Parameters
    ----------
    models : list of tuple
        List of tuples where each tuple contains the model file path and native file path.

    Returns
    -------
    results_list : list of tuple
        List of tuples containing model_id, DockQ, DockQ_F1, and fnat scores.
    """
    results_list = []
    for model_file, native_file in models:
        print(f"Processing model: {model_file}, native: {native_file}")
        model_id = os.path.basename(model_file).split(".")[0]
        model = load_PDB(model_file)
        native = load_PDB(native_file)

        chain_ids = list(model.child_dict.keys())
        print(chain_ids)
        native_chain_ids = list(native.child_dict.keys())

        if len(chain_ids) == 3:
            print(f"Model {model_id} has 3 chains: {chain_ids}")
            
            # Merge A and B chains and recalculate
            model_merged = merge_chains(model, chain_ids[:2])
            print(model_merged)
            native_merged = merge_chains(native, native_chain_ids[:2])
            chain_map_merged = {native_chain_ids[2]: chain_ids[2], "".join(native_chain_ids[:2]): "".join(chain_ids[:2])}
            results_merged, dockq_score_merged = calculate_dockq(model_merged, native_merged, chain_map_merged)
            merged_result = results_merged[list(results_merged.keys())[0]]
            # results_list.append((model_id, merged_result['DockQ'], merged_result['DockQ_F1'], merged_result['fnat']))
            results_list.append((model_id, merged_result['DockQ'], merged_result['fnat'],
                                merged_result['iRMSD'], merged_result['LRMSD'],merged_result['F1']))

        elif len(chain_ids) == 2:
            print(f"Model {model_id} has 2 chains: {chain_ids}")
            # Assume there are only 2 chains
            chain_map = {native_chain_ids[0]: chain_ids[0], native_chain_ids[1]: chain_ids[1]}
            results, dockq_score = calculate_dockq(model, native, chain_map)
            results_list.append((model_id, results[list(results.keys())[0]]['DockQ'], results[list(results.keys())[0]]['fnat'],
                                results[list(results.keys())[0]]['iRMSD'], results[list(results.keys())[0]]['LRMSD'],
                                results[list(results.keys())[0]]['F1']))

        else:
            print(f"Model {model_id} does not have 2 or 3 chains: {chain_ids}, skipping.")
            continue
    return results_list

def save_results_to_csv(results, filename):
    print(f"Saving results to CSV file: {filename}")
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # writer.writerow(['model_id', 'DockQ', 'DockQ_F1', 'fnat'])
        writer.writerow(['model_id', 'DockQ', 'fnat',"iRMSD","LRMSD","F1"])

        for model_id, dockq, fnat, irms, lrms, f1 in results:
            # print(f"Writing row: {model_id}, {dockq}, {dockq_f1}, {fnat}")
            # writer.writerow([model_id, dockq, dockq_f1, fnat])
            print(f"Writing row: {model_id}, {dockq}, {fnat}, {irms},{lrms},{f1}")
            writer.writerow([model_id, dockq, fnat, irms, lrms,f1])   

def main(directory, original_directory):
    """
    Main function to find PDB files, process models, and save results to a CSV file.

    Parameters
    ----------
    directory : str
        Path to the directory containing PDB files to be analyzed.
    original_directory : str
        Path to the directory containing original PDB files for comparison.

    Returns
    -------                                                                                                                                                                                                                                                                                                                                    
    None
    """
    pdb_files = glob.glob(f"{directory}/*_unrelaxed_rank_00*.pdb")
    if not pdb_files:
        print(f"No PDB files found in directory: {directory}")
        return

    print(f"Found PDB files: {pdb_files}")
    models = []

    for pdb_file in pdb_files:
        pdb_id_chains = os.path.basename(pdb_file).split('_unrelaxed')[0] # for af2.3
        pdb_id_chains = "_".join(pdb_id_chains.split("_")[:-1])
        print(f"Searching for original files for: {pdb_id_chains}")
        original_pdb_files = glob.glob(f"{original_directory}/{pdb_id_chains}.pdb")  # Find matching original PDB files
        print(f"Found original PDB files: {original_pdb_files} for {pdb_file}")
        if not original_pdb_files:
            print(f"No matching original PDB file found for {pdb_file}, skipping.")
            continue
        for original_pdb_file in original_pdb_files:
            models.append((pdb_file, original_pdb_file))
            print(f"Adding model-native pair: {pdb_file}, {original_pdb_file}")

    if not models:
        print("No valid model-native pairs found. Exiting.")
        return
    os.chdir(result_path)
    results = process_models(models)
    save_results_to_csv(results, str(pdb_id_chains)+'_dockq_fnat_scores.csv')
    for model_id, dockq, fnat, irms, lrms, f1 in results:
            # print(f"Writing row: {model_id}, {dockq}, {dockq_f1}, {fnat}")
            # writer.writerow([model_id, dockq, dockq_f1, fnat])
        print(f"Writing row: {model_id}, {dockq}, {fnat}, {irms},{lrms},{f1}")

/home/emel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set directory paths
import pandas as pd
import glob
import os

for i in complex_list:
    fpath = f"{folder_path}/{i}"
    directory = os.path.expanduser(fpath)
    original_directory = os.path.expanduser(original_directory)
    main(directory, original_directory)

def combine_csv_files(result_path, output_file=None):
    # Find all CSV files in the folder
    csv_files = glob.glob(os.path.join(result_path, "*dockq_fnat_scores.csv"))
    
    # Read and combine all CSVs
    df_list = [pd.read_csv(file) for file in csv_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Save to CSV if an output file is provided
    if output_file:
        combined_df.to_csv(output_file, index=False)
        print(f"Combined CSV saved to {output_file}")
    return combined_df
combined_df = combine_csv_files(result_path, dockq_output)

Found PDB files: ['/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_002_alphafold2_multimer_v3_model_1_seed_002.pdb', '/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_004_alphafold2_multimer_v3_model_2_seed_002.pdb', '/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_003_alphafold2_multimer_v3_model_4_seed_002.pdb', '/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_001_alphafold2_multimer_v3_model_3_seed_002.pdb', '/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_005_alphafold2_multimer_v3_model_5_seed_002.pdb']
Searching for original files for: 7bbj_H_L_A
Found original PDB files: ['/media/emel/WD_3tb/Ab-Ag_benchmark_bolt

In [4]:
### pdockq2 functions

from Bio.PDB import PDBIO
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Selection import unfold_entities
import numpy as np
import sys,os
import argparse
import pickle
import itertools
import pandas as pd
from scipy.optimize import curve_fit

def retrieve_IFplddt(structure, chain1, chain2_lst, max_dist):
    chain_lst = list(chain1) + chain2_lst
    ifplddt = []
    contact_chain_lst = []
    for res1 in structure[0][chain1]:
        for chain2 in chain2_lst:
            count = 0
            for res2 in structure[0][chain2]:
                if res1.has_id('CA') and res2.has_id('CA'):
                   dis = abs(res1['CA']-res2['CA'])
                   ## add criteria to filter out disorder res
                   if dis <= max_dist:
                      ifplddt.append(res1['CA'].get_bfactor())
                      count += 1
                elif res1.has_id('CB') and res2.has_id('CB'):
                   dis = abs(res1['CB']-res2['CB'])
                   if dis <= max_dist:
                      ifplddt.append(res1['CB'].get_bfactor())
                      count += 1
            if count > 0:
              contact_chain_lst.append(chain2)
    contact_chain_lst = sorted(list(set(contact_chain_lst)))   
    if len(ifplddt)>0:
       IF_plddt_avg = np.mean(ifplddt)
    else:
       IF_plddt_avg = 0
    return IF_plddt_avg, contact_chain_lst


def retrieve_IFPAEinter(structure, paeMat, contact_lst, max_dist):
    chain_lst = [x.id for x in structure[0]]
    seqlen = [len(x) for x in structure[0]]
    ifch1_col=[]
    ifch2_col=[]
    ch1_lst=[]
    ch2_lst=[]
    ifpae_avg = []
    d=10
    for ch1_idx in range(len(chain_lst)):
      idx = chain_lst.index(chain_lst[ch1_idx])
      ch1_sta=sum(seqlen[:idx])
      ch1_end=ch1_sta+seqlen[idx]
      ifpae_col = []   
      for contact_ch in contact_lst[ch1_idx]:
        index = chain_lst.index(contact_ch)
        ch_sta = sum(seqlen[:index])
        ch_end = ch_sta+seqlen[index]
        paeMat = np.array(paeMat)
        remain_paeMatrix = paeMat[ch1_sta:ch1_end,ch_sta:ch_end]
        mat_x = -1
        for res1 in structure[0][chain_lst[ch1_idx]]:
          mat_x += 1
          mat_y = -1
          for res2 in structure[0][contact_ch]:
              mat_y+=1
              if res1['CA'] - res2['CA'] <=max_dist:
                 ifpae_col.append(remain_paeMatrix[mat_x,mat_y])
      if not ifpae_col:
        ifpae_avg.append(0)
      else:
        norm_if_interpae=np.mean(1/(1+(np.array(ifpae_col)/d)**2))
        ifpae_avg.append(norm_if_interpae)
    return ifpae_avg

def calc_pmidockq(ifpae_norm, ifplddt):
    df = pd.DataFrame()
    df['ifpae_norm'] = ifpae_norm
    df['ifplddt'] = ifplddt
    df['prot'] = df.ifpae_norm*df.ifplddt
    fitpopt = [1.31034849e+00, 8.47326239e+01, 7.47157696e-02, 5.01886443e-03] ## from orignal fit function  
    df['pmidockq'] = sigmoid(df.prot.values, *fitpopt)
    return df

def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

def process_pdb_file(pdb_file, json_file, distance, file_id, chains_part=""): 
    pdbp = PDBParser(QUIET=True)
    structure = pdbp.get_structure('', pdb_file)
    chains = [chain.id for chain in structure[0]]
    remain_contact_lst = []
    plddt_lst = []
    for idx in range(len(chains)):
        chain2_lst = list(set(chains)-set(chains[idx]))
        IF_plddt, contact_lst = retrieve_IFplddt(structure, chains[idx], chain2_lst, distance)
        plddt_lst.append(IF_plddt)
        remain_contact_lst.append(contact_lst)
    pae_data = pd.read_json(json_file, lines=True)
    avgif_pae = retrieve_IFPAEinter(structure, pae_data["pae"][0], remain_contact_lst, distance)
    res = calc_pmidockq(avgif_pae, plddt_lst)
    pdb_id = os.path.basename(pdb_file).split('_')[0]
    result = {
        "model_id" : pdb_file,
        "pdb_id": file_id,
        "pdb_id_with_chains": '{0}_{1}'.format(pdb_id, chains_part),
        "ipae_norm_ag": res['ifpae_norm'].tolist()[-1],
        "ipae_norm_avg": np.mean(res['ifpae_norm']), 
        "iplddt_ag": res['ifplddt'].tolist()[-1],
        "iplddt_avg": np.mean(res['ifplddt']), 
        "pDockQ2_ag": res['pmidockq'].tolist()[-1],
        "pDockQ2_avg": np.mean(res['pmidockq'])}
    return result

In [5]:
import os
import glob
import pandas as pd

def find_matching_json(pdb_file, directory):
    pdb_file_basename = os.path.basename(pdb_file)
    parts = pdb_file_basename.split('_')
    pdb_id = parts[0]
    chains_part = "_".join(parts[1:parts.index([p for p in parts if p.startswith('unrelaxed')][0])])
    pattern = f"{pdb_id}_{chains_part}_scores_rank_00*.json"
    json_files = glob.glob(os.path.join(directory, pattern))
    return json_files[0] if json_files else print(pattern + " not found")

def run_processing(directory):
    pdb_files = glob.glob(os.path.join(directory, "*_unrelaxed_rank_00*.pdb"))
    results = []   
    for pdb_file in pdb_files:
        base_name = pdb_file.rsplit('_', 4)[0]
        pdb_file_basename = os.path.basename(pdb_file)
        parts = pdb_file_basename.split('_')
        pdb_id = parts[0]
        chains_part = "_".join(parts[1:parts.index([p for p in parts if p.startswith('unrelaxed')][0])])
        json_file_name = find_matching_json(pdb_file, directory)
        if os.path.exists(json_file_name):
            print(f"{pdb_id}_{chains_part}")
            file_id = f"{pdb_id}_{chains_part}"
            result = process_pdb_file(pdb_file, json_file_name, 8, pdb_id, chains_part)  # Adjust Ca-ca distance
            print(f"Result for {pdb_file}: {result}")  
            results.append(result)
        else:
            print(f"JSON file for {pdb_file} not found.")
            return
    if results:
        df = pd.DataFrame(results)
        os.chdir(result_path)
        csv_file_path = str(file_id)+"_pdockq2_fit.csv"
        df.to_csv(csv_file_path, index=False)
        print(f"Data saved to {csv_file_path}")
    else:
        print("No results to save.")

### run for all predictions
for i in complex_list:
    fpath = f"{folder_path}/{i}"
    directory = fpath
    run_processing(directory)   

def combine_csv_files(result_path, output_file=None):
    csv_files = glob.glob(os.path.join(result_path, "*_pdockq2_fit.csv"))
    df_list = [pd.read_csv(file) for file in csv_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    if output_file:
        combined_df.to_csv(output_file, index=False)
        print(f"Combined CSV saved to {output_file}")
    return combined_df
    
combined_df = combine_csv_files(result_path, pdockq_output)

7bbj_H_L_A_af2
Result for /media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_002_alphafold2_multimer_v3_model_1_seed_002.pdb: {'model_id': '/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_002_alphafold2_multimer_v3_model_1_seed_002.pdb', 'pdb_id': '7bbj', 'pdb_id_with_chains': '7bbj_H_L_A_af2', 'ipae_norm_ag': 0.8612688988419595, 'ipae_norm_avg': 0.9243954669705808, 'iplddt_ag': 86.10689655172413, 'iplddt_avg': 92.0079914501952, 'pDockQ2_ag': 0.41411006197380795, 'pDockQ2_avg': 0.6751138482261085}
7bbj_H_L_A_af2
Result for /media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_004_alphafold2_multimer_v3_model_2_seed_002.pdb: {'model_id': '/media/emel/WD_3tb/Ab-Ag_benchmark_boltz_chai/paper/part1/recycle20/af2_trial/7bbj_H_L_A_af2/7bbj_H_L_A_af2_unrelaxed_rank_004_alphafold2_

In [6]:
#extract log metrics
import pandas as pd
import re

def parse_log_file(log_file):
    data = []
    query_pdb = None

    with open(log_file, 'r') as f:
        for line in f:
            line = line.strip()
            match_query = re.search(r'Query \d+/\d+: (\S+) ', line)
            if match_query:
                query_pdb = match_query.group(1)
            match_model = re.search(r'(rank_\d+_alphafold2_multimer_v3_model_\d+_seed_\d+) '
                                    r'pLDDT=([\d.]+) pTM=([\d.]+) ipTM=([\d.]+) actifpTM=([\d.]+)', line)
            if match_model and query_pdb:
                model_name = match_model.group(1)
                scores = list(map(float, match_model.groups()[1:]))  
                data.append([query_pdb, model_name] + scores)
    df = pd.DataFrame(data, columns=["Query_PDB", "Model_Name", "pLDDT", "pTM", "ipTM","actifpTM"])
    return df

In [15]:
### combine all files

x=[]
for i in complex_list:
    # print(f"{folder_path}/{i}")
    df = parse_log_file(f"{folder_path}/{i}/log.txt")
    # print(df)
    x.append(df)
dfy = pd.concat(x)

dfy['Model_Name']=dfy['Model_Name'].str.replace("rank","unrelaxed_rank")
dfy['model_id'] = dfy['Query_PDB'] + "_"+dfy['Model_Name']
dfy['model_id']
dfy=dfy[["model_id","pLDDT","pTM","ipTM","actifpTM"]]
print(list(dfy))
# print(dfy)
df = pd.read_csv(f"{result_path}/{pdockq_output}")
df['model_id']=df['model_id'].str.split("/").str[-1]
df['model_id']=df['model_id'].str.replace(".pdb","")
# print(df)
df2 = pd.read_csv(f"{result_path}/{dockq_output}")
# print(df2)
dfx = pd.merge(df,df2,on="model_id")

df_fin = pd.merge(dfx,dfy,on="model_id")
df_fin=df_fin.drop_duplicates()
print(list(df))
df_fin["pdb_id_wchains"]=df_fin["pdb_id_with_chains"].str.replace("_af2","")
df_fin=df_fin[['model_id', 'pdb_id', 'pdb_id_wchains', 'ipae_norm_ag', 'ipae_norm_avg', 'iplddt_ag', 'iplddt_avg', 
               'pDockQ2_ag', 'pDockQ2_avg','pLDDT', 'pTM', 'ipTM', 'actifpTM']]

# AntiConf = 0.3pDockQ2_ag + 0.7pTM
df_fin["AntiConf"] = 0.3* df_fin["pDockQ2_ag"] + 0.7 * df_fin["pTM"]
df_fin.to_csv(f"{result_path}/{combined}",index=False)
df_fin

['model_id', 'pLDDT', 'pTM', 'ipTM', 'actifpTM']
['model_id', 'pdb_id', 'pdb_id_with_chains', 'ipae_norm_ag', 'ipae_norm_avg', 'iplddt_ag', 'iplddt_avg', 'pDockQ2_ag', 'pDockQ2_avg']


,model_id,pdb_id,pdb_id_wchains,ipae_norm_ag,ipae_norm_avg,iplddt_ag,iplddt_avg,pDockQ2_ag,pDockQ2_avg,pLDDT,pTM,ipTM,actifpTM,AntiConf
0,7bbj_H_L_A_af2_unrelaxed_rank_002_alphafold2_m...,7bbj,7bbj_H_L_A,0.861269,0.924395,86.106897,92.007991,0.414110,0.675114,95.4,0.890,0.830,0.980,0.747233
1,7bbj_H_L_A_af2_unrelaxed_rank_004_alphafold2_m...,7bbj,7bbj_H_L_A,0.740115,0.805950,87.822206,80.562738,0.249095,0.304681,90.6,0.599,0.391,0.977,0.494028
2,7bbj_H_L_A_af2_unrelaxed_rank_003_alphafold2_m...,7bbj,7bbj_H_L_A,0.756375,0.809916,81.379508,79.979791,0.202031,0.283529,90.1,0.608,0.409,0.979,0.486209
3,7bbj_H_L_A_af2_unrelaxed_rank_001_alphafold2_m...,7bbj,7bbj_H_L_A,0.869900,0.928233,83.085833,90.652597,0.375568,0.656616,95.4,0.907,0.837,0.980,0.747570
4,7bbj_H_L_A_af2_unrelaxed_rank_005_alphafold2_m...,7bbj,7bbj_H_L_A,0.789836,0.852181,84.351273,80.400739,0.274134,0.348193,90.2,0.601,0.383,0.961,0.502940
5,7ar0_B_A_af2_unrelaxed_rank_002_alphafold2_mul...,7ar0,7ar0_B_A,0.258368,0.243610,82.066757,72.755270,0.016296,0.014091,86.3,0.712,0.252,0.318,0.503289
6,7ar0_B_A_af2_unrelaxed_rank_003_alphafold2_mul...,7ar0,7ar0_B_A,0.244612,0.218178,73.537000,64.291250,0.013904,0.012018,84.9,0.703,0.210,0.281,0.496271
7,7ar0_B_A_af2_unrelaxed_rank_005_alphafold2_mul...,7ar0,7ar0_B_A,0.274432,0.253049,78.444444,62.184444,0.016570,0.013367,84.2,0.679,0.124,0.150,0.480271
8,7ar0_B_A_af2_unrelaxed_rank_001_alphafold2_mul...,7ar0,7ar0_B_A,0.272847,0.273753,76.924231,69.381731,0.016118,0.014692,86.0,0.719,0.306,0.427,0.508135
9,7ar0_B_A_af2_unrelaxed_rank_004_alphafold2_mul...,7ar0,7ar0_B_A,0.269535,0.245756,78.254545,61.542727,0.016203,0.013056,82.8,0.670,0.129,0.162,0.473861
